In [1]:

# Your data
X = [-2, -1, 0, 1, 2]
Y = [1, 2, 3, 3, 4]

# Number of data points
n = len(X)

# Step 1: Calculate sums
sum_x = sum(X)
sum_y = sum(Y)
sum_xy = sum(X[i] * Y[i] for i in range(n))
sum_x2 = sum(X[i] ** 2 for i in range(n))

# Step 2: Calculate SLOPE (m)
m = (n * sum_xy - sum_x * sum_y) / (n * sum_x2 - (sum_x ** 2))

# Step 3: Calculate INTERCEPT (b)
b = (sum_y - m * sum_x) / n

# Show results
print("Slope (m):", m)
print("Intercept (b):", b)

# Step 4: Predict
def predict(x):
    return m * x + b

print("Prediction for x = 15:", predict(15))

Slope (m): 0.7
Intercept (b): 2.6
Prediction for x = 15: 13.1


In [4]:

# Your data
X = [1, 2, 3, 4, 5]
Y = [3, 4,5, 2, 1]

# Number of data points
n = len(X)

# Step 1: Calculate sums
sum_x = sum(X)
sum_y = sum(Y)
sum_xy = sum(X[i] * Y[i] for i in range(n))
sum_x2 = sum(X[i] ** 2 for i in range(n))

# Step 2: Calculate SLOPE (m)
b = (n * sum_xy - sum_x * sum_y) / (n * sum_x2 - (sum_x ** 2))

# Step 3: Calculate INTERCEPT (b)
a = (sum_y - m * sum_x) / n

# Show results
print("Slope (b):", b)
print("Intercept (a):", a)

# Step 4: Predict
def predict(x):
    return b * x + b

print("Prediction for x = 15:", predict(15))

Slope (b): -0.6
Intercept (a): 4.8
Prediction for x = 15: -9.6
